## sentdex - 딥러닝 Pt.7

<h3> 목표 : RNN과 친해지자</h3>
<hr>

In [1]:
import warnings
warnings.filterwarnings(action = "ignore")

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM #CuDNNLSTM

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape) # 60000개의 샘플 이미지
print(x_train[0].shape)

(60000, 28, 28)
(28, 28)


**GPU 할당 문제**

In [4]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))

**정규화!**

In [5]:
x_train = x_train/255.0
x_test = x_test/255.0

In [6]:
model = Sequential()

# Layer1 - LSTM (Input) 셀
model.add(LSTM(128, input_shape = (x_train.shape[1:]), activation = "relu",
          return_sequences = True))
model.add(Dropout(0.2)) # 오버피팅 방지를 위한 드롭아웃 추가

# Layer2 - LSTM 셀
model.add(LSTM(128, activation = "relu"))
model.add(Dropout(0.2))

# Layer3 - Dense layer
model.add(Dense(32, activation = "relu"))
model.add(Dropout(0.2))

# Layer4 - Dense (Output) layer
model.add(Dense(10, activation = "softmax"))

opt = tf.keras.optimizers.Adam(lr = 1e-3, decay = 1e-5) # decay로 점점 스텝 사이즈 감소

model.compile(loss = "sparse_categorical_crossentropy",
             optimizer = opt,
             metrics = ['accuracy'])

model.fit(x_train, y_train, epochs = 3, validation_data = (x_test, y_test))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/3
60000/60000 [==============================] - 160s 3ms/sample - loss: 0.6328 - acc: 0.7918 - val_loss: 0.1741 - val_acc: 0.9473
Epoch 2/3
60000/60000 [==============================] - 159s 3ms/sample - loss: 0.1700 - acc: 0.9537 - val_loss: 0.0998 - val_acc: 0.9709
Epoch 3/3
60000/60000 [==============================] - 159s 3ms/sample - loss: 0.1157 - acc: 0.9694 - val_loss: 0.0755 - val_acc: 0.9776


<hr><hr><hr><br>
<h4>CuDNNLSTM 사용을 위해선 tensorflow 말고 tensorflow-gpu로 바꿔서 깔 것!</h4>
<h4>CuDNNLSTM : 훨씬 빠름!!!!</h4>

In [7]:
from tensorflow.keras.layers import CuDNNLSTM

In [8]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape) # 60000개 샘플 이미지
print(x_train[0].shape)

x_train = x_train/255.0
x_test = x_test/255.0

model = Sequential()

model.add(CuDNNLSTM(128, input_shape = (x_train.shape[1:]),
          return_sequences = True))
model.add(Dropout(0.2))

model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))

model.add(Dense(32, activation = "relu"))
model.add(Dropout(0.2))

model.add(Dense(10, activation = "softmax"))

opt = tf.keras.optimizers.Adam(lr = 1e-3, decay = 1e-5)

model.compile(loss = "sparse_categorical_crossentropy",
             optimizer = opt,
             metrics = ['accuracy'])

model.fit(x_train, y_train, epochs = 3, validation_data = (x_test, y_test))

(60000, 28, 28)
(28, 28)
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 16s 260us/sample - loss: 0.4027 - acc: 0.8786 - val_loss: 0.1177 - val_acc: 0.9657
Epoch 2/3
60000/60000 [==============================] - 14s 238us/sample - loss: 0.1268 - acc: 0.9666 - val_loss: 0.0944 - val_acc: 0.9750
Epoch 3/3
60000/60000 [==============================] - 14s 238us/sample - loss: 0.0886 - acc: 0.9769 - val_loss: 0.0565 - val_acc: 0.9835


Someday Imma solve the error...   
Tensorflow Official Website on GPU : https://www.tensorflow.org/install/gpu   
Cuda Installation : https://docs.nvidia.com/deeplearning/cudnn/install-guide/index.html#download-windows